# Setup

Import a few common modules

In [23]:
#CODE for points = 1
# import sklearn, numpy, os
import numpy as np
import sklearn
import os
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"
# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

%load_ext tensorboard



# to make this notebook's output stable across runs
np.random.seed(42)


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


# Vanishing/Exploding Gradients Problem

In [24]:
def logit(z):
    return 1 / (1 + np.exp(-z))

In [25]:
z = np.linspace(-5, 5, 200)

## Xavier and He Initialization

In [26]:
#CODE for points = 1
# set activation to relu and kernel initializer to he_normal
[name for name in dir(keras.initializers) if not name.startswith("_")]
keras.layers.Dense(10, activation="relu", kernel_initializer="he_normal")

### Leaky ReLU

In [27]:
#CODE for points = 1

# remember α is the hyperparameter that defines how much the function “leaks”
# set the value of alpha, use the value typically set

def leaky_relu(z, alpha=0.01):
    return np.maximum(alpha*z, z)

Let's train a neural network on Fashion MNIST using the Leaky ReLU:

In [28]:
#CODE for points = 1
#load MNIST dataset from keras

(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

In [29]:
#CODE for points = 1

# initialize kernel_initializer to "he_normal" and activation function to softmax

tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(),
    keras.layers.Dense(100, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(),
    keras.layers.Dense(10, activation="softmax")
])

In [30]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

In [31]:
history = model.fit(X_train, y_train, epochs=5,
                    validation_data=(X_valid, y_valid))

Epoch 1/5
1719/1719 [==============================] - 5s 3ms/step - loss: 1.2819 - accuracy: 0.6229 - val_loss: 0.8886 - val_accuracy: 0.7160
Epoch 2/5
1719/1719 [==============================] - 5s 3ms/step - loss: 0.7955 - accuracy: 0.7361 - val_loss: 0.7130 - val_accuracy: 0.7656
Epoch 3/5
1719/1719 [==============================] - 5s 3ms/step - loss: 0.6816 - accuracy: 0.7721 - val_loss: 0.6427 - val_accuracy: 0.7900
Epoch 4/5
1719/1719 [==============================] - 5s 3ms/step - loss: 0.6217 - accuracy: 0.7944 - val_loss: 0.5900 - val_accuracy: 0.8064
Epoch 5/5
1719/1719 [==============================] - 5s 3ms/step - loss: 0.5832 - accuracy: 0.8074 - val_loss: 0.5582 - val_accuracy: 0.8200


Now look at what happens if we try to use the ReLU activation function instead:

In [32]:
np.random.seed(42)
tf.random.set_seed(42)

In [33]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28, 28]))

model.add(keras.layers.Dense(300, activation="relu", kernel_initializer="he_normal"))

for layer in range(99):
    model.add(keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal"))

model.add(keras.layers.Dense(10, activation="softmax"))

In [34]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

In [35]:
history = model.fit(X_train, y_train, epochs=5,
                    validation_data=(X_valid, y_valid))

Epoch 1/5
1719/1719 [==============================] - 26s 13ms/step - loss: 1.9651 - accuracy: 0.2648 - val_loss: 1.9584 - val_accuracy: 0.2052
Epoch 2/5
1719/1719 [==============================] - 22s 13ms/step - loss: 1.3499 - accuracy: 0.4457 - val_loss: 0.9428 - val_accuracy: 0.6000
Epoch 3/5
1719/1719 [==============================] - 22s 13ms/step - loss: 0.9219 - accuracy: 0.6075 - val_loss: 1.1507 - val_accuracy: 0.5130
Epoch 4/5
1719/1719 [==============================] - 22s 13ms/step - loss: 0.9883 - accuracy: 0.6011 - val_loss: 1.5673 - val_accuracy: 0.3342
Epoch 5/5
1719/1719 [==============================] - 22s 13ms/step - loss: 0.9605 - accuracy: 0.6095 - val_loss: 0.8842 - val_accuracy: 0.6760


Not great at all, we suffered from the vanishing/exploding gradients problem.

# Batch Normalization

In [36]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation="softmax")
])

In [37]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_9 (Flatten)         (None, 784)               0         
                                                                 
 batch_normalization (BatchN  (None, 784)              3136      
 ormalization)                                                   
                                                                 
 dense_226 (Dense)           (None, 300)               235500    
                                                                 
 batch_normalization_1 (Batc  (None, 300)              1200      
 hNormalization)                                                 
                                                                 
 dense_227 (Dense)           (None, 100)               30100     
                                                                 
 batch_normalization_2 (Batc  (None, 100)             

In [38]:
bn1 = model.layers[1]
[(var.name, var.trainable) for var in bn1.variables]

[('batch_normalization/gamma:0', True),
 ('batch_normalization/beta:0', True),
 ('batch_normalization/moving_mean:0', False),
 ('batch_normalization/moving_variance:0', False)]

In [39]:
#bn1.updates #deprecated

In [40]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

In [41]:
history = model.fit(X_train, y_train, epochs=5,
                    validation_data=(X_valid, y_valid))

Epoch 1/5
1719/1719 [==============================] - 8s 4ms/step - loss: 0.8750 - accuracy: 0.7124 - val_loss: 0.5525 - val_accuracy: 0.8228
Epoch 2/5
1719/1719 [==============================] - 7s 4ms/step - loss: 0.5753 - accuracy: 0.8031 - val_loss: 0.4725 - val_accuracy: 0.8472
Epoch 3/5
1719/1719 [==============================] - 8s 4ms/step - loss: 0.5190 - accuracy: 0.8204 - val_loss: 0.4375 - val_accuracy: 0.8548
Epoch 4/5
1719/1719 [==============================] - 7s 4ms/step - loss: 0.4827 - accuracy: 0.8324 - val_loss: 0.4152 - val_accuracy: 0.8596
Epoch 5/5
1719/1719 [==============================] - 7s 4ms/step - loss: 0.4566 - accuracy: 0.8410 - val_loss: 0.3997 - val_accuracy: 0.8644


Sometimes applying BN before the activation function works better (there's a debate on this topic). Moreover, the layer before a `BatchNormalization` layer does not need to have bias terms, since the `BatchNormalization` layer some as well, it would be a waste of parameters, so you can set `use_bias=False` when creating those layers:

In [42]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("relu"),
    keras.layers.Dense(100, use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("relu"),
    keras.layers.Dense(10, activation="softmax")
])

In [43]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

In [44]:
history = model.fit(X_train, y_train, epochs=5,
                    validation_data=(X_valid, y_valid))

Epoch 1/5
1719/1719 [==============================] - 9s 5ms/step - loss: 1.0317 - accuracy: 0.6757 - val_loss: 0.6767 - val_accuracy: 0.7812
Epoch 2/5
1719/1719 [==============================] - 7s 4ms/step - loss: 0.6790 - accuracy: 0.7793 - val_loss: 0.5566 - val_accuracy: 0.8180
Epoch 3/5
1719/1719 [==============================] - 8s 4ms/step - loss: 0.5960 - accuracy: 0.8037 - val_loss: 0.5007 - val_accuracy: 0.8362
Epoch 4/5
1719/1719 [==============================] - 7s 4ms/step - loss: 0.5447 - accuracy: 0.8193 - val_loss: 0.4666 - val_accuracy: 0.8448
Epoch 5/5
1719/1719 [==============================] - 8s 4ms/step - loss: 0.5109 - accuracy: 0.8279 - val_loss: 0.4434 - val_accuracy: 0.8532


# Faster Optimizers

## Momentum optimization

In [45]:
#CODE for points = 1
# initialize lr and momentum to typical values

optimizer = keras.optimizers.SGD(learning_rate=0.001, momentum=0.9)

## Nesterov Accelerated Gradient

In [46]:
#CODE for points = 1
# initialize lr and momentum to typical values. Set nesterov so that it is used

optimizer = keras.optimizers.SGD(learning_rate=0.001, momentum=0.9, nesterov=True)

# Avoiding Overfitting Through Regularization

## $\ell_1$ and $\ell_2$ regularization

In [47]:
#CODE for points = 0.5

# Use syntax for assigning l2 regularization with a factor 0.01 as given here - https://keras.io/api/layers/regularizers/

layer = keras.layers.Dense(100, activation="elu",
                           kernel_initializer="he_normal",
                           kernel_regularizer=keras.regularizers.l2(0.01))

In [48]:
#CODE for points = 1

# Use syntax for assigning l2 regularization with a factor 0.01 as given here - https://keras.io/api/layers/regularizers/
# nadam optimizer

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="elu",
                       kernel_initializer="he_normal",
                       kernel_regularizer=keras.regularizers.l2(0.01)),
    keras.layers.Dense(100, activation="elu",
                       kernel_initializer="he_normal",
                       kernel_regularizer=keras.regularizers.l2(0.01)),
    keras.layers.Dense(10, activation="softmax",
                       kernel_regularizer=keras.regularizers.l2(0.01))
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
n_epochs = 2
history = model.fit(X_train, y_train, epochs=n_epochs,
                    validation_data=(X_valid, y_valid))

Epoch 1/2
1719/1719 [==============================] - 11s 6ms/step - loss: 1.5638 - accuracy: 0.7780 - val_loss: 1.0068 - val_accuracy: 0.7690
Epoch 2/2
1719/1719 [==============================] - 10s 6ms/step - loss: 0.9056 - accuracy: 0.7945 - val_loss: 0.8504 - val_accuracy: 0.8172


In [49]:
#CODE for points = .5

# Use syntax for assigning l2 regularization with a factor 0.01 as given here - https://keras.io/api/layers/regularizers/

from functools import partial

RegularizedDense = partial(keras.layers.Dense,
                           activation="elu",
                           kernel_initializer="he_normal",
                           kernel_regularizer=keras.regularizers.l2(0.01))

In [50]:
#CODE for points = 1

# activation function initialized as softmax
# nadam optimizer


model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    RegularizedDense(300),
    RegularizedDense(100),
    RegularizedDense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
n_epochs = 2
history = model.fit(X_train, y_train, epochs=n_epochs,
                    validation_data=(X_valid, y_valid))


Epoch 1/2
1719/1719 [==============================] - 11s 6ms/step - loss: 1.6013 - accuracy: 0.7780 - val_loss: 1.0311 - val_accuracy: 0.7588
Epoch 2/2
1719/1719 [==============================] - 10s 6ms/step - loss: 0.9089 - accuracy: 0.7920 - val_loss: 0.8528 - val_accuracy: 0.8148
